In [ ]:
# deep learning with PyTorch: Neural Style Transfer
# set Google Colab runtime

# !pip install torch torchvision
# !git clone https://github.com/parth1620/Project-NST.git

# loading VSG-19 pretrained model using model subpackage in PyTorch
import torch
from torchvision import models

vgg = models.vgg19(pretrained = True)

# prints features and classifiers and their layers
# for this project, we don't use the classifiers, as we extract the content and style from the layers in the features
print(vgg)

vgg = vgg.features
# see only the features component
print(vgg)

# parse the gradient, use the pretrained weight without updating it
# freeze the model so no gradient computation occurs in the training loop
for parameters in vgg.parameters():
  parameters.requires_grad_(False)

# check and move variables and model to GPU if available, otherwise to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print to check what device was set to
print(device)
vgg.to(device)
